## Pydeck
Pydeck is a high-level Python wrapper for Deck.gl, a powerful WebGL-powered library for visual exploratory data analysis of large datasets. Pydeck is built on top of Deck.gl and provides a simple interface to create complex visualizations with minimal code.

### Advanced Visualization Effects in Pydeck 0.9.1

Pydeck 0.9.1 introduces enhanced lighting, shading, and animation effects, making geospatial visualizations more realistic and interactive. These improvements allow users to create more immersive 3D visualizations, particularly for applications like urban modeling, environmental simulations, and infrastructure planning.

In [6]:
"""
TextLayer
========

Names of various public transit stops within Leipzig plotted at the location of that stop
"""
# !pip install pydeck
import pydeck as pdk
from pydeck.types import String
import pandas as pd

TEXT_LAYER_DATA = "/Users/ulrike_imac_air/projects/maps/pydeck/LeipzigStops.json"  # noqa
df = pd.read_json(TEXT_LAYER_DATA)
df['coordinates'] = df[['longitude', 'latitude']].values.tolist()

# Define a layer to display on a map
layer = pdk.Layer(
    "TextLayer",
    df,
    pickable=True,
    get_position="coordinates",
    get_text="name",
    get_size=8,
    get_color=[0, 0, 0],
    get_angle=0,
    # Note that string constants in pydeck are explicitly passed as strings
    # This distinguishes them from columns in a data set
    get_text_anchor=String("middle"),
    get_alignment_baseline=String("center"),
)

# Set the viewport location
view_state = pdk.ViewState(latitude=51.345693, longitude=12.380429, zoom=12, bearing=0, pitch=45)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{name}\n{address}"},
    map_style=pdk.map_styles.CARTO_LIGHT,
    # map_style=pdk.map_styles.road,
)

r.show()
r.to_html("text_layerLeipzig.html")

### Expanded Compatibility with Pandas and GeoPandas in Pydeck 0.9.1

Working with geographical data stored in dataframes is made simpler with Pydeck 0.9.1, which improves interoperability with Pandas and GeoPandas. For data scientists and GIS specialists using Python-based geospatial analysis, this upgrade simplifies the process of turning tabular spatial data into interactive maps.

In [ ]:
"""
PathLayer
=========

Locations of the public transport lines, connecting each station to its two nearest neighbors.
"""

import pandas as pd
import pydeck as pdk
from math import radians, sin, cos, sqrt, atan2

DATA_URL = "/Users/ulrike_imac_air/projects/maps/pydeck/LeipzigStops.json"
df = pd.read_json(DATA_URL)

def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of earth in kilometers
    return distance

def find_two_closest_stations(station, stations):
    distances = []
    for other_station in stations:
        if station['id'] != other_station['id']:
            dist = calculate_distance(station['latitude'], station['longitude'],
                                      other_station['latitude'], other_station['longitude'])
            distances.append((dist, other_station))

    distances.sort(key=lambda x: x[0])
    return [distances[0][1], distances[1][1]] if len(distances) >= 2 else []


path_data = []
# Create a list of station dicts for faster processing
stations_list = df.to_dict('records')
for stop in stations_list:
    closest_stations = find_two_closest_stations(stop, stations_list)
    for closest_stop in closest_stations:
        path_data.append({
            'name': f"{stop['name']} to {closest_stop['name']}",
            'path': [
                [stop['longitude'], stop['latitude']],
                [closest_stop['longitude'], closest_stop['latitude']]
            ]
        })

df_paths = pd.DataFrame(path_data)

view_state = pdk.ViewState(latitude=51.345693, longitude=12.380429, zoom=10)

layer = pdk.Layer(
    "PathLayer",
    df_paths,
    pickable=True,
    get_color=[160, 32, 240],
    width_scale=20,
    width_min_pixels=2,
    get_path="path",
    get_width=5,
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}"}, map_style=pdk.map_styles.CARTO_DARK)
r.show()
r.to_html("path_layerLeipzig.html")
